**Zad 10**

Utworzyć tabele M1(K INT, V VARCHAR(20)) oraz S1(K INT, MFK INT, V VARCHAR(20)), gdzie K jest kluczem głównym, a MFK jest kluczem obcym do tabeli M. Następnie utworzyć M2 oraz S2 z tą różnicą, że klucz główny M2 składa się z dwóch kolumn: K1 oraz K2, a S2 ma odpowiednio dopasowany klucz obcy. Dodać trochę danych i sprawdzić, że relacja klucza obcego istotnie działa. Ostatecznie, dodać klauzule ON UPDATE oraz ON DELETE i zademonstrować działanie relacji klucza obcego dla wartości NO ACTION, SET NULL oraz CASCADE.

## Table creation

In [32]:
IF NOT EXISTS (SELECT * from sysobjects where name='M1' and xtype='U')
    CREATE TABLE M1 (
	    K INT NOT NULL,
	    V VARCHAR(20),
	    PRIMARY KEY(K)
    );
GO

IF NOT EXISTS (SELECT * from sysobjects where name='S1' and xtype='U')
    CREATE TABLE S1 (
	    K INT NOT NULL,
	    MFK INT,
	    V VARCHAR(20),
	    PRIMARY KEY(K),
	    FOREIGN KEY (MFK) REFERENCES M1(K)
    );
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.157

In [4]:
IF NOT EXISTS (SELECT * from sysobjects where name='M2' and xtype='U')
    CREATE TABLE M2 (
	    K1 INT NOT NULL,
	    K2 INT NOT NULL,
	    V VARCHAR(20),
	    PRIMARY KEY(K1, K2)
    );
GO

IF NOT EXISTS (SELECT * from sysobjects where name='S2' and xtype='U')
    CREATE TABLE S2 (
	    K INT NOT NULL,
	    MFK1 INT,
	    MFK2 INT,
	    V VARCHAR(20),
	    PRIMARY KEY(K),
	    FOREIGN KEY (MFK1, MFK2) REFERENCES M2(K1, K2)
    );
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.165

### TESTS

In [33]:
INSERT INTO M1
VALUES (2, 'abc'),
       (3, 'def')

SELECT * FROM M1

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.065

K,V
2,abc
3,def


In [34]:
-- error -> no key 42 in M
INSERT INTO S1
VALUES (41, 42, 'a')

: Msg 547, Level 16, State 0, Line 2
The INSERT statement conflicted with the FOREIGN KEY constraint "FK__S1__MFK__5CA1C101". The conflict occurred in database "kpabd-test", table "dbo.M1", column 'K'.

The statement has been terminated.

Total execution time: 00:00:00.058

In [35]:
INSERT INTO S1
VALUES (41, 3, 'a')

SELECT * FROM S1

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.061

K,MFK,V
41,3,a


In [12]:
UPDATE M1
SET K = 42
WHERE K = 3

: Msg 547, Level 16, State 0, Line 1
The UPDATE statement conflicted with the REFERENCE constraint "FK__S1__MFK__5224328E". The conflict occurred in database "kpabd-test", table "dbo.S1", column 'MFK'.

The statement has been terminated.

Total execution time: 00:00:00.146

In [14]:
INSERT INTO M2
VALUES (2, 3, 'a'),
       (4, 5, 'b'),
       (4, 2, 'c')

SELECT * FROM M2

(3 rows affected)

(3 rows affected)

Total execution time: 00:00:00.077

K1,K2,V
2,3,a
4,2,c
4,5,b


In [18]:
INSERT INTO S2
VALUES (42, 2, 3, 'd')

SELECT * FROM S2

The statement has been terminated.

(1 row affected)

Total execution time: 00:00:00.057

K,MFK1,MFK2,V
42,2,3,d


: Msg 2627, Level 14, State 1, Line 1
Violation of PRIMARY KEY constraint 'PK__S2__3BD019B08D91D1D3'. Cannot insert duplicate key in object 'dbo.S2'. The duplicate key value is (42).

### ON UPDATE

default action -\> RESTRICT (zmiana K w M1 jest niemożliwa jeśli odnosi się do jakiegoś wiersza w S1)

NO ACTION -\> DEFAULT

SET NULL -\> set child data to NULL when parent is UPDATED 

CASCADE -\> update child data when parent is UPDATED 

SET DEFAULT -\> set child data to default when parent is UPDATED

In [58]:
-- fails because of RESTRICT
UPDATE M1
SET K = 42
WHERE K = 3

(1 row affected)

Total execution time: 00:00:00.064

In [65]:
-- fails because of RESTRICT
DELETE
FROM M1
WHERE K = 3


(1 row affected)

Total execution time: 00:00:00.059

In [63]:
ALTER TABLE S1 DROP CONSTRAINT MFK;
ALTER TABLE S1 DROP CONSTRAINT MFK_2;

: Msg 3728, Level 16, State 1, Line 2
'MFK_2' is not a constraint.

: Msg 3727, Level 16, State 0, Line 2
Could not drop constraint. See previous errors.

Total execution time: 00:00:00.068

In [57]:
ALTER TABLE S1
ADD CONSTRAINT MFK FOREIGN KEY (MFK) REFERENCES M1(K)
    ON UPDATE CASCADE

Commands completed successfully.

Total execution time: 00:00:00.064

In [64]:
ALTER TABLE S1
ADD CONSTRAINT MFK_2 FOREIGN KEY (MFK) REFERENCES M1(K)
    ON DELETE CASCADE

Commands completed successfully.

Total execution time: 00:00:00.063

In [66]:
SELECT * FROM M1

(1 row affected)

Total execution time: 00:00:00.053

K,V
2,abc


In [67]:
SELECT * FROM S1

(0 rows affected)

Total execution time: 00:00:00.053

K,MFK,V


## Cleanup

In [1]:
DROP TABLE IF EXISTS M1
GO
DROP TABLE IF EXISTS S1
GO
DROP TABLE IF EXISTS M2
GO
DROP TABLE IF EXISTS S2
GO

: Msg 3726, Level 16, State 1, Line 1
Could not drop object 'M1' because it is referenced by a FOREIGN KEY constraint.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.228